<a href="https://colab.research.google.com/github/Teopx/UTFPR/blob/main/EXERC%C3%8DCIO_AULA_1_Vis%C3%A3o%20Computacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#VISÃO COMPUTACIONAL
EXERCÍCIO - AULA 1

Discente: Cléber Fernando Paixão

Carregar Bibliotecas:

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

a) Carregue o conjunto de dados de câncer de mama (load_breast_cancer).

In [23]:
# Carregar o conjunto de dados e processa-los
data_file = '/content/wdbc.data'
names_file = '/content/wdbc.names'

# definição do nome das colunas:
column_names = [
    'ID', 'Diagnosis',
    'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean',
    'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
    'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
    'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se',
    'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst',
    'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst'
]

try:
    data_df = pd.read_csv(data_file, header=None, names=column_names)
    print(f"Arquivo '{data_file}' processado.")
except FileNotFoundError:
    print(f"Erro: Arquivo '{data_file}' não encontrado.")
    exit() # Erro! Arquivo não encontrado.

# Exibir as primeiras linhas do dataframe carregado:
print("\n--- Dados Carregados (primeiras 5 linhas) ---")
print(data_df.head())

Arquivo '/content/wdbc.data' carregado com sucesso.

--- Dados Carregados (primeiras 5 linhas) ---
         ID Diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302         M        17.99         10.38          122.80     1001.0   
1    842517         M        20.57         17.77          132.90     1326.0   
2  84300903         M        19.69         21.25          130.00     1203.0   
3  84348301         M        11.42         20.38           77.58      386.1   
4  84358402         M        20.29         14.34          135.10     1297.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280     

b) Para cada classificador, crie um pipeline com StandardScaler (quando necessário) e o classificador.

In [28]:
# Realizando a separação das features (X) e o alvo (y)
X = data_df.drop(columns=['ID', 'Diagnosis'])
y = data_df['Diagnosis']

# Alterar a referência, onde  (M=Maligno, B=Benigno) para campo numérico ( Benigno será 0; Maligno será 1)

le = LabelEncoder()
y = le.fit_transform(y)
print(f"\nAlteração de classe feita: {le.classes_} -> {np.unique(y)}")

print(f"\nFeatures alteradas (X): {X.shape}")
print(f"Alvos alterados (y): {y.shape}")
print("\n" + "="*80 + "\n")




Alteração de classe feita: ['B' 'M'] -> [0 1]

Features alteradas (X): (569, 30)
Alvos alterados (y): (569,)




c) Utilize GridSearchCV com validação cruzada estratificada (StratifiedKFold) com 10 dobras

d) Avalie e imprima os melhores hiperparâmetros encontrados e a acurácia média para cada modelo.

In [25]:
# Primeiro definir os hiperparâmetros e os modelos. Separar as acurácias conforme modelo proposto na tarefa 1:

models_params = [
    {
        'name': 'LogisticRegression',
        'pipeline': Pipeline([('scaler', StandardScaler()), ('clf', LogisticRegression(random_state=42))]),
        'params': {
            'clf__C': [0.1, 1.0, 10.0],
            'clf__solver': ['liblinear', 'lbfgs']
        }
    },
    {
        'name': 'SVC',
        'pipeline': Pipeline([('scaler', StandardScaler()), ('clf', SVC(random_state=42))]),
        'params': {
            'clf__C': [0.1, 1.0, 10.0],
            'clf__kernel': ['linear', 'rbf']
        }
    },
    {
        'name': 'KNeighborsClassifier',
        'pipeline': Pipeline([('scaler', StandardScaler()), ('clf', KNeighborsClassifier())]),
        'params': {
            'clf__n_neighbors': [3, 5, 7],
            'clf__weights': ['uniform', 'distance']
        }
    },
    {
        'name': 'RandomForestClassifier',
        'pipeline': Pipeline([('clf', RandomForestClassifier(random_state=42))]),
        'params': {
            'clf__n_estimators': [50, 100, 200],
            'clf__max_depth': [None, 10, 20]
        }
    },
    {
        'name': 'GradientBoostingClassifier',
        'pipeline': Pipeline([('clf', GradientBoostingClassifier(random_state=42))]),
        'params': {
            'clf__n_estimators': [50, 100, 200],
            'clf__learning_rate': [0.01, 0.1, 0.2]
        }
    },
    {
        'name': 'GaussianNB',
        'pipeline': Pipeline([('scaler', StandardScaler()), ('clf', GaussianNB())]),
        'params': {} # GaussianNB não trás a perda associada
    }
]

In [29]:
# Avaliação de modelo (análise cruzada)
# Interação dos modelos:

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)


print("Processando análise e treinamento dos modelos 1,2,3...\n")
for model_info in models_params:
    print(f"---")
    print(f"Iniciando treinamento do modelo: {model_info['name']}")

    grid_search = GridSearchCV(
        estimator=model_info['pipeline'],
        param_grid=model_info['params'],
        cv=cv,
        scoring='accuracy',
        n_jobs=-1, # Utilizando todos os núcleos da CPU disponíveis
        verbose=0 # Opção de poder ser ajustado para 1 ou 2 para mais detalhes durante o processo
    )
    grid_search.fit(X, y)

    print(f" ↑ hiperparâmetros: {grid_search.best_params_}")
    mean_accuracy = grid_search.cv_results_['mean_test_score'][grid_search.best_index_]
    std_accuracy = grid_search.cv_results_['std_test_score'][grid_search.best_index_]
    print(f"Acurácia média (10-Fold CV): {mean_accuracy:.4f} (+/- {std_accuracy:.4f})\n")

print("Comparação concluída!")

Processando análise e treinamento dos modelos .1.2.3

---
Iniciando treinamento do modelo: LogisticRegression
 ↑ hiperparâmetros: {'clf__C': 0.1, 'clf__solver': 'liblinear'}
Acurácia média (10-Fold CV): 0.9807 (+/- 0.0146)

---
Iniciando treinamento do modelo: SVC
 ↑ hiperparâmetros: {'clf__C': 10.0, 'clf__kernel': 'rbf'}
Acurácia média (10-Fold CV): 0.9772 (+/- 0.0176)

---
Iniciando treinamento do modelo: KNeighborsClassifier
 ↑ hiperparâmetros: {'clf__n_neighbors': 5, 'clf__weights': 'uniform'}
Acurácia média (10-Fold CV): 0.9701 (+/- 0.0176)

---
Iniciando treinamento do modelo: RandomForestClassifier
 ↑ hiperparâmetros: {'clf__max_depth': None, 'clf__n_estimators': 100}
Acurácia média (10-Fold CV): 0.9561 (+/- 0.0239)

---
Iniciando treinamento do modelo: GradientBoostingClassifier
 ↑ hiperparâmetros: {'clf__learning_rate': 0.2, 'clf__n_estimators': 200}
Acurácia média (10-Fold CV): 0.9667 (+/- 0.0241)

---
Iniciando treinamento do modelo: GaussianNB
 ↑ hiperparâmetros: {}
Acuráci